# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [34]:
!cat publications.tsv























1/1/2012	Meander hydrodynamics initiated by river restoration deflectors	Hydrological Processes	This study quantifies how river restoration structures change surface flow hydrodynamics	"Zhou, T., & Endreny, T. (2012). Meander hydrodynamics initiated by river restoration deflectors. Hydrological Processes, 26(22), 3378-3392."	HP-CFD	https://doi.org/10.1002/hyp.8352

## Import pandas

We are using the very handy pandas library for dataframes.

In [35]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [36]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,1/4/2020,Impact of climate change on water availability...,Applied Energy,This paper evaluates how projected changes in ...,NaN,AE-power-grid,https://doi.org/10.1016/j.apenergy.2020.115467
1,1/3/2020,The DOE E3SM v1. 1 biogeochemistry configurati...,Journal of Advances in Modeling Earth Systems,This paper documents the biogeochemistry confi...,NaN,E3SM-BGC,https://doi.org/10.1029/2019MS001766
2,1/2/2020,Global irrigation characteristics and effects ...,Journal of Advances in Modeling Earth Systems,This paper introduces a two_way coupled irriga...,NaN,E3SM-irrigation,https://doi.org/10.1029/2020MS002069
3,1/1/2020,The Straightening of a River Meander Leads to ...,Water,This paper was the last chapter of my PhD diss...,NaN,Water-ESF-flume,https://doi.org/10.3390/w12061680
4,1/1/2019,Roles of irrigation and reservoir operations i...,Journal of Geophysical Research: Atmosphere,This paper examines water management impact ov...,NaN,JGR-Indian-VIC-IRR-Reservoir,https://doi.org/10.1029/2019MS001870
5,1/2/2019,The DOE E3SM coupled model version 1: Descript...,Journal of Advances in Modeling Earth Systems,This paper overviews E3SM V1 at high resolutio...,NaN,JAMES-E3SM-overview-high-resolution,https://doi.org/10.1029/2019MS001870
6,1/3/2019,Flood Inundation Generation Mechanisms and The...,Journal of Geophysics Research: Atmospheres,This paper used MOSART-inundation to investiga...,NaN,JGR-Flood-Inundation,https://doi.org/10.1029/2019JD031381
7,1/4/2019,The DOE E3SM Coupled Model Version 1: Overview...,Journal of Advances in Modeling Earth Systems,This paper overviews E3SM V1 at standard resol...,"Golaz, J. C., Caldwell, P. M., Van Roekel, L. ...",JAMES-E3SM-overview,https://doi.org/10.1029/2018MS001603
8,1/5/2019,Strong influence of irrigation on water budget...,Journal of Geophysical Research: Atmospheres,This paper examines irrigation impacts over In...,NaN,JGR-Indian-VIC-IRR,https://doi.org/10.1029/2018JD029132
9,1/1/2018,Modulating factors of hydrologic exchanges in ...,Hydrological Processes,This paper investigates the major control fact...,NaN,HP-Columbia-River-CFD,https://doi.org/10.1002/hyp.13266


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [37]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [38]:
import os
for row, item in publications.iterrows():
    
    year = item.pub_date[4:8]
    md_filename = str(year) + "-" + item.url_slug + ".md"
    html_filename = str(year) + "-" + item.url_slug
        
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    # md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Link to the paper](" + item.paper_url + ")\n" 
    
    md += "\n![image](../images/papers/" + str(year) + "-" + item.url_slug + ".png)\n"
    
    # md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)
        
    item.pub_date

In [39]:
item.pub_date[4:8]

'2012'

These files are in the publications directory, one directory below where we're working from.

In [40]:
!ls ../_publications/

12-HP-CFD.md
13-WRR-Flume-Hyporheic.md
14-JHM-Drought-Monitoring.md
14-JHM-TRMM.md
16-AGU-Mono.md
16-JHM-VIC-Reservoir.md
17-GMD-CLM-PFLOTRAN.md
17-Water-iButton.md
18-ERL-Hydropower.md
18-HP-Columbia-River-CFD.md
18-HP-Snow-Gap.md
18-JHM-West-US-WM.md
18-WRR-Columbia-River-CFD.md
19-JAMES-E3SM-overview-high-resolution.md
19-JAMES-E3SM-overview.md
19-JGR-Flood-Inundation.md
19-JGR-Indian-VIC-IRR-Reservoir.md
19-JGR-Indian-VIC-IRR.md
20-AE-power-grid.md
20-E3SM-BGC.md
20-E3SM-irrigation.md
20-Water-ESF-flume.md
2012-HP-CFD.md
2013-WRR-Flume-Hyporheic.md
2014-JHM-Drought-Monitoring.md
2014-JHM-TRMM.md
2016-AGU-Mono.md
2016-JHM-VIC-Reservoir.md
2017-GMD-CLM-PFLOTRAN.md
2017-Water-iButton.md
2018-ERL-Hydropower.md
2018-HP-Columbia-River-CFD.md
2018-HP-Snow-Gap.md
2018-JHM-West-US-WM.md
2018-WRR-Columbia-River-CFD.md
2019-JAMES-E3SM-overview-high-resolution.md
2019-JAMES-E3SM-overview.md
2019-JGR-Flood-Inundation.md
2019-JGR-Indian-VIC-IRR-Reservoir.md
2019-JGR-Indian-VIC-IRR.md
2020-AE-pow

In [41]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
